In [7]:
from qiskit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit_ibm_provider import IBMProvider
import pandas as pd
import os
import json

import matplotlib.pyplot as plt
from datetime import datetime

import seaborn as sns

plt.rcParams.update({'font.size': 14, 'font.family': 'serif', 'mathtext.fontset': 'dejavuserif'})

In [8]:
service = QiskitRuntimeService(channel='ibm_quantum',
    instance='ibm-q/open/main',
    token='8c865d438487275bb939e03adfb9ebd4c3e3fe4adce788c4655b12b5a938a17de0139cc89d372c60bc9c6037496beb8377f4d84ce30c194bb908ae27047c0d09')
# IBM Quantum channel; set to default

# Mostro i backend disponibili per l'esecuzione
print(service.backends())

backend_maps = {
    index: backend.name.split('_')[1] for index, backend in enumerate(service.backends())
}

# Visualizo che i nomi siano corretti 
print(backend_maps)
# Inizializzo il backend array e la mappatura
backend_array = []
backend_map = {index: backend.name.split('_')[1] for index, backend in enumerate(service.backends(operational=True, simulator=False))}

print("Macchine disponibili:")
for index, name in backend_map.items():
    print(f"{index}: {name}")

scelta = input("Scegli un'opzione (1/2/3): 1 meno occupata, 2 tutte, 3 scegli manualmente ")

if scelta == "1":
    print("Hai scelto l'opzione 1!")
    print("Scelgo la macchina meno occupata")
    backend = service.least_busy(operational=True, simulator=False, min_num_qubits=127)
    backend_array.append(backend)
    print(f"Macchina selezionata: {backend.name}")

elif scelta == "2":
    print("Hai scelto l'opzione 2!")
    backend_array = [service.backend(f"ibm_{name}") for name in backend_map.values()]
    print("Macchine selezionate:", [backend.name for backend in backend_array])

elif scelta == "3":
    print("Hai scelto l'opzione 3!")
    scelte_utente = input("Inserisci gli indici delle macchine separate da una virgola: ")
    indici = [int(i.strip()) for i in scelte_utente.split(',') if i.strip().isdigit()]
    backend_array = []
    new_backend_map = {}
    for index in indici:
        if index in backend_map:
            class_name = f"ibm_{backend_map[index]}"
            try:
                backend = service.backend(class_name, instance='ibm-q/open/main')
                backend_array.append(backend)
                new_backend_map[len(new_backend_map)] = backend.name.split('_')[1]
            except Exception as e:
                print(f"Errore nel trovare la macchina '{class_name}': {e}")
    backend_map = new_backend_map
    print("Macchine selezionate:", [backend.name for backend in backend_array])
    print("Nuovo mapping aggiornato:", backend_map)
else:
    print("Scelta non valida!")

[<IBMBackend('ibm_brisbane')>, <IBMBackend('ibm_sherbrooke')>]
{0: 'brisbane', 1: 'sherbrooke'}
Macchine disponibili:
0: brisbane
1: sherbrooke
Hai scelto l'opzione 3!
Macchine selezionate: ['ibm_sherbrooke']
Nuovo mapping aggiornato: {0: 'sherbrooke'}


In [9]:
def normalized_fractional_hamming_intradistance(dict1, dict2):
    # Verifica se i dizionari hanno le stesse chiavi
    if set(dict1.keys()) != set(dict2.keys()):
        raise ValueError("Response must have the same lenght")
    # Calcola la somma delle differenze assolute tra i valori corrispondenti
    sum_diff = sum(abs((dict1[key] - dict2[key])) for key in dict1.keys())
    fractional_hamming_distance = sum_diff / 2 #len(dict1) perchè la somma ha come range [0,2] quindi /2 per normalizzare a range [0,1]
    return fractional_hamming_distance

In [10]:
import pickle
n_challenge = int(input("Inserisci il numero di challenge da utilizzare (10/20): "))
n_qubits = int(input("Inserisci il numero di QBITS: "))
shots= int(input("Inserisci il numero shots: "))
n_runs = int(input("Inserisci il numero di runs ")) 

# Loading from file (this is the only one file used for all the metrics)
file_name = f'./results/configuration_challenges/random_arrays_{n_challenge}.pkl'
with open(file_name, 'rb') as f:
    a_challenge, b_challenge, c_challenge, d_challenge, e_challenge, f_challenge = pickle.load(f)

print(f"Arrays loaded from '{file_name}'")

Arrays loaded from './results/configuration_challenges/random_arrays_20.pkl'


In [11]:
'''def qcirc(i):
  # Create a Quantum Circuit
  circuit = QuantumCircuit(n_qubits, n_qubits)
  #calcolo media response
  for j in range(n_qubits):
    if j%2==0:
      circuit.rx(c_challenge[i][j],j)
      circuit.ry(b_challenge[i][j],j)
      circuit.rz(a_challenge[i][j],j)
    else:
      circuit.rx(f_challenge[i][j],j)
      circuit.ry(e_challenge[i][j],j)
      circuit.rz(d_challenge[i][j],j)
      circuit.cz(j,j-1)

  for j in range(1,n_qubits-1):
    if j%2==1:
      circuit.rx(c_challenge[i][j],j)
      circuit.ry(b_challenge[i][j],j)
      circuit.rz(a_challenge[i][j],j)
    else:
      circuit.rx(f_challenge[i][j],j)
      circuit.ry(e_challenge[i][j],j)
      circuit.rz(d_challenge[i][j],j)
      circuit.cz(j,j-1)

  circuit.barrier()
  
  for j in range(n_qubits):
      if j%2==0:
        circuit.rx(c_challenge[i][j],j)
      else:
        circuit.rx(f_challenge[i][j],j)

  circuit.barrier()
  circuit.measure(range(n_qubits), range(n_qubits))
  #drawing the circuit
  name=f"circuit_real_{n_qubits}_{n_challenge}.png"
  circuit.draw('mpl',  filename=name)
  return circuit'''

'def qcirc(i):\n  # Create a Quantum Circuit\n  circuit = QuantumCircuit(n_qubits, n_qubits)\n  #calcolo media response\n  for j in range(n_qubits):\n    if j%2==0:\n      circuit.rx(c_challenge[i][j],j)\n      circuit.ry(b_challenge[i][j],j)\n      circuit.rz(a_challenge[i][j],j)\n    else:\n      circuit.rx(f_challenge[i][j],j)\n      circuit.ry(e_challenge[i][j],j)\n      circuit.rz(d_challenge[i][j],j)\n      circuit.cz(j,j-1)\n\n  for j in range(1,n_qubits-1):\n    if j%2==1:\n      circuit.rx(c_challenge[i][j],j)\n      circuit.ry(b_challenge[i][j],j)\n      circuit.rz(a_challenge[i][j],j)\n    else:\n      circuit.rx(f_challenge[i][j],j)\n      circuit.ry(e_challenge[i][j],j)\n      circuit.rz(d_challenge[i][j],j)\n      circuit.cz(j,j-1)\n\n  circuit.barrier()\n  \n  for j in range(n_qubits):\n      if j%2==0:\n        circuit.rx(c_challenge[i][j],j)\n      else:\n        circuit.rx(f_challenge[i][j],j)\n\n  circuit.barrier()\n  circuit.measure(range(n_qubits), range(n_qubits))

In [12]:
def qpuf_ibm(backend):

  job = service.job(backend)
  result = job.result()
  
  pub_result = []

  for i in range(n_challenge*n_runs):
    pub_result.append(result[i].data.c.get_counts())
    # da controllare con franco 
    #print(f" >> c output register counts: {result[i].data.c.get_counts()}")
  
  #lista delle responses formattate
  responses=[]
  for j in range(len(pub_result)):
    #iesima response
    response = {}
    for i in range(2**n_qubits):
      binary_string = format(i, '0' + str(n_qubits) + 'b')
      #se trova la combinazione copia il valore e lo divide per il numero dei shots per normalizzarlo tra 0 e 1, altrimenti mette 0
      response[binary_string] = pub_result[j].get(binary_string,0)/shots
    responses.append(response)
  return responses

def run_qcirc(i, ibm_retrive):
  r=qpuf_ibm(ibm_retrive[i])

  #cosi salvo il backend e utilizzo lo stesso per le altre metriche 
  file_path = f'./results/backend/real/{circuit_type}/backend_{backend_map[i]}_nq{n_qubits}_nc{n_challenge}_s{shots}_nr{n_runs}.json'
  with open(file_path, 'w') as json_file:
    json.dump(r, json_file)
  print(f"Backend_{backend_map[i]} salvato correttamente")

print(backend_array)
print(backend_map)
scelta = input("Scegli un'opzione (1/2/3/4/5/6/7/8): 1 Earth, 2 Mars, 3 Saturn, 4 Jupiter, 5 Venus, 6 Mercury, 7 Uranus, 8 Uranus-Titania")
if scelta == "1":
   print("Hai scelto l'opzione 1 Earth!")
   circuit_type = 'earth'
elif scelta == "2":
   print("Hai scelto l'opzione Mars!")
   circuit_type = 'mars'
elif scelta == "3":
   print("Hai scelto l'opzione Saturn") 
   circuit_type = 'saturn'
elif scelta == "4":
   print("Hai scelto l'opzione Jupiter") 
   circuit_type = 'jupiter' 
elif scelta == "5":
   print("Hai scelto l'opzione Venus") 
   circuit_type = 'venus'
elif scelta == "6":
   print("Hai scelto l'opzione Mercury") 
   circuit_type = 'mercury'
elif scelta == "7":
   print("Hai scelto l'opzione Uranus") 
   circuit_type = 'uranus'
elif scelta == "8":
   print("Hai scelto l'opzione Uranus-Titania") 
   circuit_type = 'uranusTitania'

ibm_retrive = ['d0m68ch5nwm00085j68g']

for i in range(len(backend_array)):
  run_qcirc(i, ibm_retrive)

[<IBMBackend('ibm_sherbrooke')>]
{0: 'sherbrooke'}
Hai scelto l'opzione Venus
Backend_sherbrooke salvato correttamente
